In [7]:
from pymongo import MongoClient
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import json 
import folium

In [8]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [9]:
db, coll = connectCollection('companies','companies')

In [10]:
companies = list(coll.find({"offices.city": "San Jose","founded_year": {"$lt": 2008},"offices.longitude":{"$ne":None},"offices.latitude":{"$ne": None}}))

In [11]:
print(len(companies))

89


In [12]:
print(companies[0]['offices'][0]['longitude'], companies[0]['offices'][0]['latitude'])

-121.930035 37.295005


In [13]:
set(companies[0]['offices'][0])

{'address1',
 'address2',
 'city',
 'country_code',
 'description',
 'latitude',
 'longitude',
 'state_code',
 'zip_code'}

In [48]:
companies

[{'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'permalink': 'ebay',
  'crunchbase_url': 'http://www.crunchbase.com/company/ebay',
  'homepage_url': 'http://ebay.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'eBay',
  'category_code': 'web',
  'number_of_employees': 15000,
  'founded_year': 1995,
  'founded_month': 9,
  'founded_day': 1,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'auction, bid, search, market',
  'alias_list': '',
  'email_address': '',
  'phone_number': '',
  'description': 'Online Marketplace',
  'created_at': 'Wed Jun 06 13:03:13 UTC 2007',
  'updated_at': 'Tue Sep 10 03:38:03 UTC 2013',
  'overview': '<p>Founded in 1995 in San Jose, CA, eBay connects millions of buyers and sellers globally in the world&#8217;s largest online marketplace, utilizing <a href="/company/paypal" title="PayPal" rel="nofollow">PayPal</a> to ensure secure transactions. The

In [14]:
def getLocation(company):
    location=[]
    for i in range(len(company['offices'])):        
        longitude = company['offices'][i]['longitude']
        latitude = company['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude,latitude]
        }
        location.append(loc)
    return location

In [15]:
for company in companies:
    value = {"$set": {'location':getLocation(company)}}
    coll.update_one(company,value)

In [28]:
api_key = os.getenv("KEY")
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
query = input('Search query: ') 
r = requests.get(url + 'query=' + query +
                        '&key=' + api_key) 
x = r.json() 
y = x['results']
results_maps_api = []
for i in range(len(y)): 
    print(y[i]['geometry']['location'])  
    results_maps_api.append(y[i]['geometry']['location'])

Search query: starbucks in san jose, ca
{'lat': 37.387198, 'lng': -121.861847}
{'lat': 37.320964, 'lng': -121.949113}
{'lat': 37.3233208, 'lng': -121.9140698}
{'lat': 37.383275, 'lng': -121.894485}
{'lat': 37.37322100000001, 'lng': -121.87303}
{'lat': 37.3716214, 'lng': -121.9229905}
{'lat': 37.2632213, 'lng': -121.9187767}
{'lat': 37.267176, 'lng': -121.833203}
{'lat': 37.37096289999999, 'lng': -121.9171986}
{'lat': 37.385373, 'lng': -121.927951}
{'lat': 37.3252694, 'lng': -121.8138697}
{'lat': 37.33738, 'lng': -121.888408}
{'lat': 37.3333942, 'lng': -121.887332}
{'lat': 37.322351, 'lng': -121.825122}
{'lat': 37.335283, 'lng': -121.892859}
{'lat': 37.332341, 'lng': -121.912665}
{'lat': 37.300602, 'lng': -121.771928}
{'lat': 37.341364, 'lng': -121.911577}
{'lat': 37.322475, 'lng': -121.994058}
{'lat': 37.291912, 'lng': -121.87897}


In [51]:
results_maps_api[0]['lng']

-121.861847

In [52]:
#  starbucks in san jose, ca

def getLocationMapsApi(company):
    results_json=[]
    for i in range(len(company)):        
        longitude = company[i]['lng']
        latitude = company[i]['lat']
        loc = {         
            'type':'Point',
            'coordinates':[longitude,latitude]
        }
        results_json.append(loc)
    return results_json

In [53]:
getLocationMapsApi(results_maps_api)

[{'type': 'Point', 'coordinates': [-121.861847, 37.387198]},
 {'type': 'Point', 'coordinates': [-121.949113, 37.320964]},
 {'type': 'Point', 'coordinates': [-121.9140698, 37.3233208]},
 {'type': 'Point', 'coordinates': [-121.894485, 37.383275]},
 {'type': 'Point', 'coordinates': [-121.87303, 37.37322100000001]},
 {'type': 'Point', 'coordinates': [-121.9229905, 37.3716214]},
 {'type': 'Point', 'coordinates': [-121.9187767, 37.2632213]},
 {'type': 'Point', 'coordinates': [-121.833203, 37.267176]},
 {'type': 'Point', 'coordinates': [-121.9171986, 37.37096289999999]},
 {'type': 'Point', 'coordinates': [-121.927951, 37.385373]},
 {'type': 'Point', 'coordinates': [-121.8138697, 37.3252694]},
 {'type': 'Point', 'coordinates': [-121.888408, 37.33738]},
 {'type': 'Point', 'coordinates': [-121.887332, 37.3333942]},
 {'type': 'Point', 'coordinates': [-121.825122, 37.322351]},
 {'type': 'Point', 'coordinates': [-121.892859, 37.335283]},
 {'type': 'Point', 'coordinates': [-121.912665, 37.332341]},


In [ ]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["Starbucks"]

In [54]:
for company in results_maps_api:
    value = {"$set": {'location':getLocationMapsApi(results_maps_api)}}
    coll.update_one(company,value)

In [ ]:
results_json

In [ ]:
#  Starbucks in San Jose, Ca


NameError: name 'MongoClient' is not defined

In [ ]:
starbucks_list = gmaps.places('starbucks,san jose,ca')

In [18]:
map_city=folium.Map(location=[37.3382,-121.8863], zoom_start=12)
for company in companies:
    folium.Marker(company['location'][0]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='gray'), 
                   ).add_to(map_city)
    
for result in results_list:
    folium.Marker(results[0]['lat'],results_list[result]['lng'],
                  radius=2, 
                  icon=folium.Icon(icon='cloud',color='green'),
                 ).add_to(map_city)

In [19]:
# Gray: Empresas con mas de diez años.
# Green : Starbucks.
map_city